## ae expanded & annotate network
use pfib dataset to run aberrant network analysis 20250520

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')

import os, sys
from tqdm import tqdm
import importlib
import logging
import datetime

# load ae_net source code
sourcedir = '/mnt/disk7t/xwj/axolotl_rev/gitcode'
sys.path.append(sourcedir)
# from functions import *
# %run functions.py
import functions

In [2]:
importlib.reload(functions)

<module 'functions' from '/mnt/disk7t/xwj/axolotl_rev/gitcode/functions.py'>

load packaged functions

In [3]:
# output directory
outdir = '/mnt/disk7t/xwj/axolotl_rev/output' 
os.makedirs(outdir+'/log', exist_ok=True)
os.makedirs(outdir+'/data', exist_ok=True)
os.makedirs(outdir+'/sweet', exist_ok=True)

# Defining Filename Templates:
filename_nd_template = '{outdir}/data/node_df_{sample}.txt'
filename_ed_template = '{outdir}/data/edge_df_{sample}.txt'
filename_ed_aberrant_template = '{outdir}/data/edge_df_{sample}_aberrant.txt'
filename_scatter_template = '{outdir}/data/edge_scatter_{sample}.pdf'

reading and processing different datasets related to gene expression and outlier detection.

In [4]:
## pfib423 dataset
# path to files from upstream AE detection tools
# your_dataset_file = '/media/eys/xwj/axolotl_rev/pfib_423/df_cts_gm2022_dedup.txt'
# datadir = '/mnt/disk7t/xwj/axolotl_rev/pfib_423/'
datadir = f'{sourcedir}/example'
result_baseline = {
    'cts': f'{datadir}/t0_g12543_s423.txt.gz',
    'outsingle':f'{datadir}/osg_res_1_001.txt',
    # 'outrider': f'{datadir}/outrider_res_1_001.txt',
    # 'outrider_cts': f'{datadir}/outrider_norm_cts_1_001.txt',
    'trueoutlier': f'{datadir}/df_true49.txt',
    'sampleanno': f'{datadir}/sample_annot.tsv', # all samples meta information，useful in batch removal
}
#------------------------------------------------------------------------#
file = result_baseline['sampleanno']
s_anno = pd.read_csv(file, sep="\t", index_col='RNA_ID')
# pfib423 have two types sequencing library: strand & non strand. split as two groups
# rename: stranded -> ss； non-stranded -> ns
s_anno['group'] = s_anno['STRAND_SPECIFIC'].map({True:'ss', False:'ns'})
s_anno.describe()

#------------------------------------------------------------------------#
# results of upstream tools
# method 1. outsingle
file = result_baseline['outsingle']
score = pd.read_csv(file, sep="\t", index_col=0)

if False:
    # method 2. if use OUTRIDER res
    file = result_baseline['outrider']
    score = pd.read_csv(file, sep="\t", index_col=0)
    score = score.pivot_table(values='pValue',index= 'geneID', columns='sampleID')
    cts_outrider = pd.read_csv(result_baseline['outrider_cts'], sep="\t", index_col=0)

#------------------------------------------------------------------------#
# known outliers
outlier = pd.read_csv(result_baseline['trueoutlier'], sep="\t", index_col=None)
outlier['group'] = s_anno.loc[ outlier['Sample'],'group'].values

#------------------------------------------------------------------------#
# read count table
file = result_baseline['cts']
cts = pd.read_csv(file, sep="\t", index_col=0)
exp = cts.transform(lambda x: np.log(x+1))


In [5]:
outlier.head()

,Sample,Sex,Gene,RNAdefects,TableName,group
0,R80184,M,ALDH18A1,"AE, MAE",S2,ns
1,R60537,M,ATP6AP1,"AE, Var",S2,ss
2,R62943,F,MICOS13,"AE, AS",S2,ss
3,R96820,F,CLPP,"AE, AS",S2,ns
4,R77611,F,DLD,"AE, MAE",S2,ss


In [6]:
s_anno.head()

,INDIVIDUAL_ID,TISSUE,SEX,AFFECTED,ICD_10,PAIRED_END,STRAND_SPECIFIC,group
RNA_ID,,,,,,,,
R0052,p11688,Fibroblast,Female,True,E88,True,False,ns
R0053,p48517,Fibroblast,Male,True,E88,True,False,ns
R19907,p46808,Fibroblast,Male,True,E88,True,False,ns
R10581,p51250,Fibroblast,Female,True,E88,True,False,ns
R0055,p5413,Fibroblast,Female,True,E88,True,False,ns


In [7]:
# ------------------------------
#  Dataset preprocess
# ------------------------------
# dataset split by sequencing library type (stranded/non-stranded) as ss/ns batches
batches = ['ss','ns']

# filename dict
dict_filename = dict.fromkeys(batches)

for group in batches:
    
    # expression matrix 
    sample_list = s_anno.query('group == @group').index.tolist()
    expfile = f'{outdir}/df_exp_{group}{len(sample_list)}.txt'
    exp.loc[:, sample_list].to_csv( expfile, sep='\t')
    
    # score matrix
    scorefile = f'{outdir}/df_score_{group}{len(sample_list)}.txt'
    score.loc[:, sample_list].to_csv( scorefile, sep='\t')
    
    # sample meta table
    sample_list_interest = outlier.query('group == @group')['Sample']
    outlierfile = f'{outdir}/df_true_{group}{len(sample_list_interest)}.txt'
    outlier.query('Sample in @sample_list_interest').to_csv(outlierfile,sep='\t')
        
    dict_filename[group] = {}
    dict_filename[group]['exp'] = expfile
    dict_filename[group]['score'] = scorefile
    dict_filename[group]['outlier'] = outlierfile
    print(group, len(sample_list), len(sample_list_interest))
    
print(dict_filename)

ss 269 25
ns 154 24
{'ss': {'exp': '/mnt/disk7t/xwj/axolotl_rev/output/df_exp_ss269.txt', 'score': '/mnt/disk7t/xwj/axolotl_rev/output/df_score_ss269.txt', 'outlier': '/mnt/disk7t/xwj/axolotl_rev/output/df_true_ss25.txt'}, 'ns': {'exp': '/mnt/disk7t/xwj/axolotl_rev/output/df_exp_ns154.txt', 'score': '/mnt/disk7t/xwj/axolotl_rev/output/df_score_ns154.txt', 'outlier': '/mnt/disk7t/xwj/axolotl_rev/output/df_true_ns24.txt'}}


In [8]:
# ------------------------------
#  Sample Selection 
# ------------------------------

# s_list = [ 'R52016','R28774'] #from ns sample
# s_list = [ "R15264" ] #from ss sample

# manually choose one group to run. 
group = 'ns' 
# load data
file_e = dict_filename[group]['exp']
df_exp = pd.read_csv(file_e, sep="\t", index_col=0)

file_sc = dict_filename[group]['score']
df_score = pd.read_csv(file_sc, sep="\t", index_col=0)

outlierfile = dict_filename[group]['outlier']
df_outlier = pd.read_csv(outlierfile, sep="\t", index_col=0)

# select subset of samples with outliers
s_list = df_outlier['Sample'].tolist()

run aberrant network and edge analysis

In [9]:
# reload functions in case of code changes
importlib.reload(functions)
# current time format as string for log file name
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

logging.basicConfig(filename=f'{outdir}/log/ae_net_{current_time}.log', level=logging.INFO, 
                format='%(asctime)s:%(levelname)s:%(message)s')

#-----------------------------------------------------------
# prepare intermediate data
#-----------------------------------------------------------

# correlation
pcc = pd.DataFrame( np.corrcoef(df_exp), index=df_exp.index.tolist(), columns=df_exp.index.tolist())
pcc.index.name, pcc.columns.name ='gene1','gene2'
logging.info("Prepare coexpression pcc matrix")

# pcc remove duplicate, get undirected network = 0.5 min
# pcc = dict_pcc[c][ctype][p].copy()
# pcc_tri = pd.Series(pcc.stack()).reset_index().rename(columns={0:'pcc'})
# pcc_tri = pcc_tri[ pcc_tri['gene1'] < pcc_tri['gene2'] ]

pcc_tri = functions.pearson_extract_upper_triangle(pcc)
logging.info("Prepare coexpression edge table")

logging.info(f"{'DataFrame':<15} | {'Shape':<10}")
logging.info("-" * 40)
logging.info(f"{'df_outlier':<15} | {df_outlier.shape}")
logging.info(f"{'df_exp':<15} | {df_exp.shape}")
logging.info(f"{'df_score':<15} | {df_score.shape}")
logging.info(f"{'pcc':<15} | {pcc.shape}")
logging.info(f"{'pcc_tri':<15} | {pcc_tri.shape}")
logging.info("\n")

In [10]:
# input data: df_exp, df_score, df_outlier, pcc_tri, pcc
# cts = No normalize
# log = log transform
# use log(cts) or log(cpm) compute Pearsons' correlation as global coexpression network edges.
# expression table using raw counts log transformed (Optional also do count per million normalization)
c, ctype, p = 'raw', 'cts', 'log' # for method develop purpose; leave them default

thres_score = 0.0001 # threshold of ae scores
thres_q_pcc = 0.01 # in percent threshold of whole pearsons matrix 
ae_gene_max = 20  # num of top ae genes 
pcc_gene_max = 10  # num of correlated genes per ae gene; affect network size significantly
pcc_gene_min = 5 # if not have any correlated genes, take some genes below threshold.

kwargs = {'output_dir': outdir,
          'df_exp': df_exp, # df
          'df_score': df_score, # df
          'df_outlier': df_outlier, # df
          'pcc_tri': pcc_tri,# df
          'pcc': pcc, # df
          'c': c, 
          'ctype': ctype,
          'p': p,
          'thres_score': thres_score, 
          'thres_q_pcc': thres_q_pcc,
          'pcc_gene_min': pcc_gene_min, 
          'pcc_gene_max': pcc_gene_max, 
          'ae_gene_max': ae_gene_max,
          'file_e': file_e, # SWEET need absolute file path to compute sample weight
         }

In [11]:
# ------------------------------
# Part 1. construct Reference Network for datasets and all samples( ~ 0.5min )
# ------------------------------

G_int = functions.global_net(**kwargs)
logging.info("Global coexpress network of all genes")
kwargs['G_int'] = G_int     # Update kwargs

In [12]:
# ------------------------------
# Part 2. Generate AE Network for One Sample ( ~ 1min per sample)
# ------------------------------

for s in tqdm(s_list):
    logging.info(f"{s} AE network start")
    nodes, edges = functions.ae_net(s, **kwargs)
    nodes.to_csv(filename_nd_template.format(outdir=outdir, sample=s), sep='\t', header=True, index=True)
    edges.to_csv(filename_ed_template.format(outdir=outdir, sample=s), sep='\t', header=True, index=True)
    
    # identifying aberrant edges
    edges = functions.label_aberrant_edges(edges)
    edges.to_csv(filename_ed_aberrant_template.format(outdir=outdir, sample=s), sep='\t', header=True, index=True)
    # scatter plots to show aberrant edges
    fig, axs = functions.create_scatter_plot(edges)
    fig.savefig(filename_scatter_template.format(outdir=outdir, sample=s ), bbox_inches='tight')
    plt.close(fig)
    logging.info(f"{s} AE network finish")
    

  0%|                                                                                                                                                                                  | 0/24 [00:38<?, ?it/s]


In [13]:
! tail '{outdir}/log/ae_net'*

==> /mnt/disk7t/xwj/axolotl_rev/output/log/ae_net_20250519_222056.log <==
2025-05-19 22:21:34,145:INFO:prior
0.5    42
1.0    26
Name: count, dtype: int64
2025-05-19 22:21:34,146:INFO:AE net: check AE network connectivity, 2
2025-05-19 22:21:34,146:INFO:Nodes:163      Edges:223      Is Connected:True

2025-05-19 22:21:34,632:INFO:AE net: run SWEET analysis
2025-05-19 22:21:37,246:INFO:patients:{'R60537'}, index:[252], total samples:269, total genes:12543


==> /mnt/disk7t/xwj/axolotl_rev/output/log/ae_net_20250519_225237.log <==
2025-05-19 22:53:15,315:INFO:Nodes:163      Edges:223      Is Connected:True

2025-05-19 22:53:15,864:INFO:AE net: run SWEET analysis
2025-05-19 22:53:18,483:INFO:patients:{'R60537'}, index:[252], total samples:269, total genes:12543

2025-05-19 22:53:39,761:INFO:R60537 AE network finish
2025-05-20 08:05:23,084:ERROR:No such comm target registered: jupyter.widget.control
2025-05-20 08:05:23,088:WARNING:No such comm: 85a52789-9f04-4f76-8a54-0a364afee28a
2025-05-

In [14]:
# %run /public236T/test1/axolotl_rev/script/sweet_functions.py
# %run /mnt/disk7t/xwj/axolotl_rev/script/functions.py